In [ ]:
%%capture
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score
#%pip install git+https://github.com/huggingface/nlp.git@fix-bad-type-in-overflow-check

In [ ]:
QPATH = "Quantlet/4-qode2desc"

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os
if IN_COLAB:
  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}')

In [ ]:
import pickle
import json
import re
import sys
from IPython.display import display
import datetime

from tqdm import tqdm
import pandas as pd
import numpy as np
tqdm.pandas()


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

import nltk
nltk.download('punkt')
import evaluate

from sklearn.model_selection import train_test_split

In [ ]:
import importlib
import analysis_modules
importlib.reload(analysis_modules)

In [ ]:
model_name = '../5-domain-pre-training/results/checkpoint-12290'

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model = AutoModelWithLMHead.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)

In [ ]:
def generate_summary(test_samples, model, tokenizer, encoder_max_length):
    inputs = tokenizer(
        test_samples["input_sequence"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask, max_length =150)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [ ]:
def batch_tokenize_preprocess(batch,
                              tokenizer,
                              max_input_length,
                              max_output_length):

    source = batch["input_sequence"]
    target = batch["output_sequence"]

    source_tokenized = tokenizer(
        source,
        padding="max_length",
        truncation=True,
        max_length=max_input_length
    )

    target_tokenized = tokenizer(
        target,
        padding="max_length",
        truncation=True,
        max_length=max_output_length
    )

    batch = {k: v for k, v in source_tokenized.items()}

    # Ignore padding in the loss

    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]

    return batch

In [ ]:
encoder_max_length = 512
decoder_max_length = 300

In [ ]:
test_dataset = load_dataset("json",
                            data_files='test_dataset_20231007.json',
                            field="data",
                            data_dir='../../data/preprocessed/Quantlet/no_context/')


validation_data_txt = test_dataset['train']

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch,
        tokenizer=tokenizer,
        max_input_length=encoder_max_length,
        max_output_length=decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)


In [ ]:
# SUBSAMPLE FOR GENERATION BEFORE TUNING
test_samples = validation_data_txt.select(range(20))
summaries_before_tuning = generate_summary(test_samples,
                                            model,
                                            tokenizer,
                                            encoder_max_length)[1]

In [ ]:
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

references = test_samples['output_sequence']
predictions = summaries_before_tuning
results = rouge.compute(predictions=predictions, references=references)

bleu = bleu.compute(predictions=predictions, references=references)

In [ ]:
validation_data_txt['output_sequence']

In [ ]:
predictions

In [ ]:
bleu

In [ ]:
results

In [ ]:
test_samples['output_sequence']

In [ ]:
summaries_before_tuning